In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

#essential imports
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

#scraping imports
import requests
from bs4 import BeautifulSoup

#plotting imports
%matplotlib inline
import matplotlib.pyplot as plt

#String matching
import re

#date
import datetime as dt

# helpers
from matching_helpers import *

In [2]:
cps = 4 #controls per subjects

# Matching CDs/Vynils and dead musicians

## 1) Cleaning the meta data generated with Amazon API

Original data set was corrupted by symbols in the descriptions and artists' names. A function was used to replace those symbols by empty strings and the required features were selected. 
The required features are artist and asin as it allows to link artist with its products and then its reviews.

Artists names are treated in a specific way, so that the lenght is checked to avoid inconsistent names, caracters are corrected to lower and spaces at the begining and end are avoided, in order to increase probability of matching the names with related products.

The description is kept to improve the matching between list of subjects' names and their corresponding products. Description was formatised to lower in a way slightly different from artists names because they don't required to check the length and spaces (it was better to don't truncate it).

In [3]:
# LOAD DATA 

features_meta = ['asin', 'artist', 'description']

musics_products = pd.read_csv(
        'DATA/metadata_processed/meta_CDs_and_Vinyl_processed(v2).csv',
        encoding = 'ISO-8859-1',
        low_memory=False)

musics_products = musics_products[features_meta] #select the required features


# CLEAN DATA

musics_products = df_rm_punctuation(musics_products) #remove undesired ponctuation like brakets and quotations marks 
#in the whole data

musics_products = musics_products[(musics_products['artist'].isnull()==False) &\
                                  (musics_products["artist"]!='')]#remove nans and empty rows

musics_products['description'] = musics_products['description'].apply(lambda s : str(s).lower()) #format description 
musics_products['artist'] = musics_products['artist'].apply(cleanstr) #format names 

musics_products.head()

,asin,artist,description
0,0001501348,ron kenoly,"lenny leblanc,alex acuna,justo almario,tom bro..."
2,0005123909,cedarmont kids,18 music videos for kids: do your ears hang lo...
5,0005134188,cedarmont kids,vhs tape
7,0026197898,the sudden passion,"when you first hear the sudden passion,you d p..."
18,0307141950,golden books,"spring has come,so little critter and dad are ..."


## 2) Cleaning and filtering the dead names data

Dead artists names were scrapped from Wikipedia. Thez are selected to keep only musicians with the corresponding birth date and death date (description was not selected). Names are formated to be lower caracters without ponctuation this is done to favorise the matching between a dead artist and its derived amazon products.

In [4]:
deaths = pd.read_csv('DATA/deaths.csv', low_memory=False, encoding="ISO-8859-1")

In [5]:
musician_features = ['clname', 'Birth Date', 'Death Date']

dead_musicians = deaths[deaths['Musician']==True].reset_index()
dead_musicians['clname'] = dead_musicians['Name'].map(cleanstr) #this function applies a lower
#transformartion and removes symbols 

dead_musicians =dead_musicians[musician_features]
dead_musicians.head()

,clname,Birth Date,Death Date
0,paul delph,1957,1996-05-21
1,jacob druckman,1928,1996-05-24
2,bradley nowell,1968,1996-05-25
3,pilar lorengar,1928,1996-06-02
4,ella fitzgerald,1917,1996-06-15


## 3) Build controls list

The control list is constructed as follows: 

1) From the metadata, a list of artists is constructed and cleaned in the same way as the dead artists (lower caracters and ponctuation is removed), then duplicates are filtered out

2) Names are "cleaned" to avoid keeping names of troups (groups of music players) to keep only single interprets. This is done by targeting names containing common english words from a library [https://github.com/dolph/dictionary (popular.txt)] as well as checking the size of the name and control If it contains any number.

Since we have a lot of controls, we can be really strict when selecting them. 

3) Artists belonging to the list of subjects (e.g dead between 1996 to 2014) are filtered out from the list of controls

In [6]:
'''
Step 1) Building the list of all musicians names by using the metadata completed with Amazon API,
cleaning for symbols, formating to lower, checking for length (>1) and removing duplicates
'''
musicians_names = [splitstr(troup) for troup in list(musics_products['artist'])] #replace separation symbols (&, /, :) 
#by  a coma then split on comas: this is done because troups are cited in the artists column of metada

musicians_names = [cleanstr(elem) for elem in np.concatenate(musicians_names)] #clean for symbols, remaining comas
# and format to lower, also check the size >1 and drop spaces at end and begining of full name

musicians_names = list(set(musicians_names)) #remove duplicates
musicians_names = musicians_names[1:]

print(musicians_names[:5])

['heimdall', 'karmacoda', 'garland jeffreys', 'fitzell', 'michael bradford']


In [7]:
'''Step 2) Removing groups from the controls lists to keep only single interprets. 
'''
def read_txt(path) :#Returns the lines of a .txt file, with '\n' characters removed 
    
    file = open(path, "r")
    tmplist = file.read().split("\n")# the last line is an empty line 
    return tmplist[:len(tmplist)-1]

def is_name(name): #Returns true if the name does not contain any common english word or digit '''
    
    if len(name.split()) <= 1 : 
        return False
    
    for word in name.split(' ') :
            if word in englishwords or any(char.isdigit() for char in word):
                return False
    return True


englishwords = read_txt('DATA/englishwords.txt') # this data was obtained
#from https://github.com/dolph/dictionary (popular.txt)

single_interprets = [elem for elem in musicians_names if is_name(elem)]

print(single_interprets[:5])
print(len(musicians_names), ' reduced to ', len(single_interprets))

['michael bradford', 'planeta imaginario', 'oascr pappano', 'hamed nikpay', 'tex williams']
162575  reduced to  41308


In [8]:
'''Step 3) Filter out subjects' names from the list of controls
'''

controls = pd.DataFrame(single_interprets, columns = ['clname']) #initialize controls

for idx, name in enumerate(dead_musicians['clname'].tolist()): # for each subject we target rows corresponding to
    # it in the controls list: this is done to double check with str.contains
    # since the list was really dirty at the begining and 
    # it is possible that some names are still composed of various names
    
    mapp = controls['clname'].str.contains(name)
    if mapp.any():
        controls = controls[mapp==False] 
        
    if idx % 100 == 0 : print('current iteration: ',idx)
        

controls['Birth Date'] = pd.Series() # add NaNs in the fields that exist only for subjects
controls['Death Date'] = pd.Series()

print('number of controls: ', len(controls))
print('number of subjects: ', len(dead_musicians))

current iteration:  0
current iteration:  100
current iteration:  200
current iteration:  300
current iteration:  400
number of controls:  41038
number of subjects:  424


## 4) Matching dead musicians and meta data

Now the datas are cleaned it is possible to match each artist name with the asins of its derived products.
The algorithm iterates over dead artists names and target rows in the metada artist and description fields if the current name is contained. 

To construct the final data, a submatching data for each dead artists is build and duplicates the fields of death date, birth date and name of the musician.

Then it is appended to a global data for subjects. 

In [9]:
''' Match dead artists and ASINs '''

pd.options.mode.chained_assignment = None # a command to avoid printing warnings
meta_dead_musicians = pd.DataFrame() # initialize the data that will contain names and
#corresponding ASINs of derived products

for idx, musician in enumerate(list(dead_musicians['clname'])):
    
    match = pd.DataFrame() #subdata of matched asins for the current name
    
    match = musics_products[ \
                (musics_products['artist'].str.contains(musician)==True) | 
                (musics_products['description'].str.contains(musician)==True) \
            ].drop('artist',axis=1)
    
    match['name'] = musician #duplicate the artist features
    match['death date'] = dead_musicians['Death Date'].get_value(idx)
    match['birth date'] = dead_musicians['Birth Date'].get_value(idx)
    
    meta_dead_musicians = meta_dead_musicians.append(match)
    
    if idx %100 == 0: 
        print('{} in {} done'.format(idx, len(dead_musicians['clname'])))

0 in 424 done
100 in 424 done
200 in 424 done
300 in 424 done
400 in 424 done


In [10]:
print('{} of artists matched out of {}'.format(len(meta_dead_musicians.groupby('name')),
                                                   len(dead_musicians['clname'].index)))
print('size of matched data : {}'.format(len(meta_dead_musicians)))
meta_dead_musicians.head()

368 of artists matched out of 424
size of matched data : 26173


,asin,description,name,death date,birth date
345352,B000KB9Y6U,1984 atlantic records - film starring jon crye...,paul delph,1996-05-21,1957
324962,B000EU1H44,"in 1950,the columbia university music departme...",jacob druckman,1996-05-24,1928
347572,B000LP4O1K,aur&#xc3;&#xa9;ole: a halo; a light or luminou...,jacob druckman,1996-05-24,1928
47783,B000002OZS,for all his tattoos and bulked-up frat-boy per...,bradley nowell,1996-05-25,1968
232762,B00008AY6K,san diegos slightly stoopid showcase their cre...,bradley nowell,1996-05-25,1968


## 5) Subsample control musicians and match meta data

Now we are going to match the controls with their coresponding asins. But as the number of controls is huge they are
randomly subsampled before. Nevertheless to keep enough controls to match them properly with subjects, it is required to have more controls than subjects. The ratio controls to subjects is pre-defined in the variable "cps".

In [11]:
from numpy.random import randint


def subsample_df (df, nbWanted) :
    assert nbWanted < len(df)
    randomIndex = randint(0, len(df)-1, nbWanted)
    
    return df.loc[randomIndex]

sub_controls = subsample_df(controls, cps*len(dead_musicians))
sub_controls = sub_controls.dropna(how='all')

print('controls reduced from {} to {}'.format(len(sub_controls.index),len(controls.index)))
sub_controls.head(3)

controls reduced from 1690 to 41038


,clname,Birth Date,Death Date
11730,orch svizzera italiana,NaN,NaN
6819,danny malando,NaN,NaN
33059,kv narayanaswamy,NaN,NaN


Once subsampled, the controls are matched with their corresponding ASINs in the same way as subjects.

This cell takes roughly 10 minutes to run for a cps of 4. 

In [12]:
''' Match controls and ASINs'''

pd.options.mode.chained_assignment = None
meta_control_musicians = pd.DataFrame()

for idx, musician in enumerate(list(sub_controls['clname'])):
    
    match = pd.DataFrame() 
    
    match = musics_products[(musics_products['artist'].str.contains(musician)==True)| (musics_products['description'].str.contains(musician)==True)].drop('artist',axis=1)
        
    match['name'] = musician
    match['death date'] = 'unknown'
    match['birth date'] = 'unknown'
    
    meta_control_musicians = meta_control_musicians.append(match)
    
    if idx %100 == 0: print('{} out of {}'.format(idx, len(sub_controls['clname'])))

0 out of 1690
100 out of 1690
200 out of 1690
300 out of 1690
400 out of 1690
500 out of 1690
600 out of 1690
700 out of 1690
800 out of 1690
900 out of 1690
1000 out of 1690
1100 out of 1690
1200 out of 1690
1300 out of 1690
1400 out of 1690
1500 out of 1690
1600 out of 1690


In [13]:
print('number of artists matched out of 1684 :', meta_control_musicians.groupby('name').count())
print('size of matched data :',len(meta_control_musicians))
meta_control_musicians.head(3)

number of artists matched out of 1684 :                        asin  description  death date  birth date
name                                                            
a piazzolla               4            4           4           4
aaron berofsky            1            1           1           1
aaron livingston          3            3           3           3
ab quintanilla iii        8            8           8           8
abdel halim hafez         6            6           6           6
ac gomes                  1            1           1           1
acker bilk mr             1            1           1           1
adam gorightly            1            1           1           1
adam michna               1            1           1           1
adele addison             3            3           3           3
adele linsalata           3            3           3           3
adi yeshaya               1            1           1           1
ahbijit vaghani           1            1          

,asin,description,name,death date,birth date
356275,B000O78KWO,"vai dvd 4402 music of verdi,cil&#xe8;a,saint-s...",orch svizzera italiana,unknown,unknown
341214,B000IFRWGM,nan,danny malando,unknown,unknown
189590,B00005HT6B,cd album,kv narayanaswamy,unknown,unknown
467496,B0087YKZKQ,1 sarasija nabha - pada varnam - mayamalavagou...,kv narayanaswamy,unknown,unknown
407646,B001UJSWA0,nan,stefka popangelova,unknown,unknown


## 6) Cleaning reviews data from Amazon:  CDs and Vinyls

Reviews are cleaned to have review times formatable to date time and also they are pre-filtered with the asins that were matched to subjects and controls in order to limit future computations.

In [22]:
import gzip
def gz_to_dataframe(datapath, filename):
    def parse(path): 
        g = gzip.open(path, 'rb') 
        for l in g: 
            yield eval(l) 
    def getDF(path): 
        i = 0 
        df = {} 
        for d in parse(path): 
            df[i] = d 
            i += 1 
        return pd.DataFrame.from_dict(df, orient='index') 
    return getDF(datapath+filename)

In [23]:
datapath = 'DATA/review/'
filename = 'reviews_CDs_and_Vinyl.json.gz'

reviews_df = gz_to_dataframe(datapath, filename)
print(reviews_df.shape)

(3749004, 9)


In [16]:
# build asins list
all_musicians_asins = pd.concat((meta_dead_musicians['asin'], meta_control_musicians['asin'])) 


def clean_reviews(review_df):
    
    filtered_reviews = reviews_df[reviews_df['asin'].isin(all_musicians_asins)]

    filtered_reviews = filtered_reviews[['asin','reviewText','summary','reviewTime']] 
    filtered_reviews['reviewTime'] = list(pd.to_datetime(filtered_reviews['reviewTime']
                                         .str.replace(',','').str.replace(' ','-')))
    
    return filtered_reviews

In [17]:
# filter based on asins
filtered_reviews = clean_reviews(reviews_df)

print('Filtering reduced size from {} to {}'.format(len(reviews_df.index),len(filtered_reviews.index)))
filtered_reviews.set_index(['asin', 'reviewTime']).head()

Filtering reduced size from 3749004 to 339385


reviewText  \
asin       reviewTime                                                      
073890015X 2000-01-29  i just kicked its just that simple if you were...   
           2006-06-02  I'm really tired by all this "metal" stuff. It...   
           2013-09-11  ok I guess a little over 2 hours was not enoug...   
           2000-07-16  I saw the show live, and enjoyed it very much....   
           2000-03-10  Everyone badmouthing this tape b/c it is a far...   

                                                                 summary  
asin       reviewTime                                                     
073890015X 2000-01-29                                         i loved it  
           2006-06-02  in case my previous review gets edited for con...  
           2013-09-11                            great late 90's concert  
           2000-07-16                      great show, terrible coverage  
           2000-03-10      Not Peace and Love...But Still Worth The Look

## 7) Matching meta/dead data with reviews data

Using the same procedure as for matching the artists' names and the metada asins, the asins from previously matched data are matched with their corresponding reviews.

Again, a matched subdata is constructed with duplicated artist features (name, death date and birth date) and appended to a global data with all the matchings. 

In [18]:
pd.options.mode.chained_assignment = None  

def matching_meta_reviews(filtered_reviews, meta_in):
    
    processed_reviews = pd.DataFrame()
    meta = meta_in.reset_index() #reset fields to get the correct features by indexing the asins 
    # recall that each asin was matched with an artist and thus with its corresponding features
    
    for idx, asin in enumerate(meta['asin']):
        
        match_reviews = pd.DataFrame()
        match_reviews = filtered_reviews[filtered_reviews['asin']==asin] #a strict equal sign is enough since asins 
        # are perfectly formated from the sources
        
        match_reviews['name'] = meta['name'].get_value(idx)
        match_reviews['death date'] = meta['death date'].get_value(idx)
        match_reviews['birth date'] = meta['birth date'].get_value(idx)

        processed_reviews = processed_reviews.append(match_reviews)

        if idx %1000 == 0: 
            print('{} in {} done'.format(idx, len(meta['asin'])))
        
    return processed_reviews

Now we can match reviews with the dead actors asins:

In [19]:
processed_reviews_sbj = pd.DataFrame(matching_meta_reviews(filtered_reviews, meta_dead_musicians))
processed_reviews_sbj.head()

0 in 26173 done
1000 in 26173 done
2000 in 26173 done
3000 in 26173 done
4000 in 26173 done
5000 in 26173 done
6000 in 26173 done
7000 in 26173 done
8000 in 26173 done
9000 in 26173 done
10000 in 26173 done
11000 in 26173 done
12000 in 26173 done
13000 in 26173 done
14000 in 26173 done
15000 in 26173 done
16000 in 26173 done
17000 in 26173 done
18000 in 26173 done
19000 in 26173 done
20000 in 26173 done
21000 in 26173 done
22000 in 26173 done
23000 in 26173 done
24000 in 26173 done
25000 in 26173 done
26000 in 26173 done


,asin,reviewText,summary,reviewTime,name,death date,birth date
2685909,B000KB9Y6U,I bought this item because it did not say vyna...,Should be more descriptive in the title.,2013-05-23,paul delph,1996-05-21,1957
2685910,B000KB9Y6U,Chrissy Faith is wonderful I must say!! What ...,You should listen this!!,2013-11-09,paul delph,1996-05-21,1957
2685911,B000KB9Y6U,this product came quickly and there were no is...,great music,2013-04-26,paul delph,1996-05-21,1957
2549964,B000EU1H44,To put it short - the sounds are very explorat...,Enchanting,2008-07-11,jacob druckman,1996-05-24,1928
2696697,B000LP4O1K,This is a delightful recording. It's lite enou...,Pleasing,2011-05-19,jacob druckman,1996-05-24,1928


The same method is applied to control set. 

In [20]:
processed_reviews_ctrls = pd.DataFrame(matching_meta_reviews(filtered_reviews, meta_control_musicians))
processed_reviews_ctrls.head()

0 in 8170 done
1000 in 8170 done
2000 in 8170 done
3000 in 8170 done
4000 in 8170 done
5000 in 8170 done
6000 in 8170 done
7000 in 8170 done
8000 in 8170 done


,asin,reviewText,summary,reviewTime,name,death date,birth date
2742726,B000O78KWO,This 1991 concert given by soprano Grace Bumbr...,In Regal Splendor,2008-11-02,orch svizzera italiana,unknown,unknown
2660418,B000IFRWGM,We listen to this uplifting music once a week....,"Beautiful music. Uplifting, as good as it gets!",2010-12-24,danny malando,unknown,unknown
1488413,B00005HT6B,A great CD - the Raghuvara (Pantuvarali) is s...,Vintage KV Naryanaswamy and Palghat Raghu,2010-12-06,kv narayanaswamy,unknown,unknown
3479266,B0087YKZKQ,"KV Narayanaswamy, knows simply as KVN, is a li...",A Lion Amongst Classical Indian Music Vocalists,2014-02-02,kv narayanaswamy,unknown,unknown
3056694,B001UJSWA0,*Recorded in Sofia (1988) this is a 2009 disco...,Real Russian Poetry...,2011-04-10,stefka popangelova,unknown,unknown


## Save files

In [24]:
processed_reviews_sbj.to_csv('DATA/music_reviews_matched_'+str(cps)+'cps.csv')

In [25]:
processed_reviews_ctrls.to_csv('DATA/music_reviews_matched_'+str(cps)+'cps_ctrls.csv')